In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance
import warnings
warnings.filterwarnings("ignore")

# ------------------------------
# Configuration: set your file paths here
# ------------------------------
DATASETS = {
    # D7 – Kaggle: Exploring Risk Factors for Cardiovascular Disease
    # Columns in your upload: gender (1=Male, 2=Female), cardio (0/1)
    "D7": {
        "path": "D7.csv",
        "gender_col": "gender",
        "label_col": "cardio",
        "gender_map": {1: 1, 2: 0},  # 1=Male->1, 2=Female->0
    },

    # D58 – Kaggle: Diabetes Health Indicators (BRFSS 2015)
    # Typical columns: Sex (0=Female, 1=Male), Diabetes_binary (0/1)
    "D58": {
        "path": "D58.csv",
        "gender_col": "Sex",
        "label_col": "Diabetes_binary",
        "gender_map": {0: 0, 1: 1},  # 0=Female->0, 1=Male->1
    },

    # D73 – UCI: Sepsis Survival minimal clinical records
    # You set: sex_0male_1female (0=Male,1=Female), hospital_outcome_1alive_0dead (0/1)
    "D73": {
        "path": "D73.csv",
        "gender_col": "sex_0male_1female",
        "label_col": "hospital_outcome_1alive_0dead",
        "gender_map": {0: 1, 1: 0},  # 0=Male->1, 1=Female->0
    },
}

# ------------------------------
# Helpers to detect/standardize columns
# ------------------------------
GENDER_CANDIDATES = ["male", "sex", "gender", "Sex", "Gender", "Male"]
LABEL_CANDIDATES  = [
    "TenYearCHD", "tenyearchd", "target", "Target", "Outcome", "outcome",
    "CHD", "label", "Label", "Survival", "survival", "DEATH_EVENT", "Diabetes_binary",
    "diabetes_binary", "diabetes", "Diabetes", "SepsisLabel", "sepsislabel", "sepsis"
]

STR_TRUE  = {"1","true","yes","y","t","pos","positive"}
STR_MALE  = {"male","m"}
STR_FEMALE= {"female","f"}

def find_column(df: pd.DataFrame, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    return None

def coerce_binary(series: pd.Series, positive_hint=None):
    """
    Coerce a gender/label series into {0,1}.
    For gender: returns (is_male: 1=male, 0=female).
    For label: returns (is_positive: 1=positive, 0=negative).
    """
    s = series.copy()

    # If numeric 0/1 already:
    if pd.api.types.is_numeric_dtype(s):
        uniq = set(pd.unique(s.dropna()))
        if uniq.issubset({0,1}):
            return s.astype(int)

    # String-like
    s = s.astype(str).str.strip().str.lower()

    if positive_hint == "male":
        return s.apply(lambda v: 1 if v in STR_MALE or v == "1" else (0 if v in STR_FEMALE or v == "0" else np.nan)).astype("float")
    elif positive_hint == "positive":
        return s.apply(lambda v: 1 if (v in STR_TRUE) else
                              (0 if (v in {"0","neg","negative","no","n","false","f"}) else np.nan)).astype("float")

    # Heuristic: if values look like M/F or male/female
    if s.isin(STR_MALE | STR_FEMALE).mean() > 0.5:
        return s.apply(lambda v: 1 if v in STR_MALE else (0 if v in STR_FEMALE else np.nan)).astype("float")

    # Otherwise try numeric strings
    return s.apply(lambda v: 1 if v in {"1"} else (0 if v in {"0"} else np.nan)).astype("float")

def infer_binary_label(df: pd.DataFrame, exclude_cols=set()):
    """
    Attempt to find a binary label column (0/1) distinct from gender.
    Preference: known candidates; otherwise any column with exactly two unique numeric values {0,1}.
    """
    # Try candidate names first
    for cand in LABEL_CANDIDATES:
        col = find_column(df, [cand])
        if col and col not in exclude_cols:
            coerced = coerce_binary(df[col], positive_hint="positive")
            if set(pd.unique(coerced.dropna())).issubset({0.0,1.0}):
                return col, coerced.astype(int)

    # Fallback: scan columns
    for col in df.columns:
        if col in exclude_cols:
            continue
        ser = df[col]
        if pd.api.types.is_numeric_dtype(ser):
            uniq = set(pd.unique(ser.dropna()))
            if uniq.issubset({0,1}) and len(uniq) == 2:
                return col, ser.astype(int)

        # try coercion
        coerced = coerce_binary(ser, positive_hint="positive")
        uniq = set(pd.unique(coerced.dropna()))
        if uniq.issubset({0.0,1.0}) and len(uniq) == 2:
            return col, coerced.astype(int)

    raise ValueError("Could not infer a binary label column; please set label_col explicitly in DATASETS.")

def get_gender_and_label(df: pd.DataFrame, gender_col=None, label_col=None, gender_map=None):
    # Gender
    gcol = gender_col or find_column(df, GENDER_CANDIDATES)
    if gcol is None:
        raise ValueError("Could not find a gender column; set 'gender_col' in DATASETS.")

    if gender_map is not None:
        gender01 = df[gcol].map(gender_map)
        if gender01.isna().any():
            raise ValueError(f"Gender mapping failed for column '{gcol}'. Check values and 'gender_map'.")
        gender01 = gender01.astype(int)
    else:
        gender = df[gcol]
        gender01 = coerce_binary(gender, positive_hint="male").astype("float")
        if gender01.isna().mean() > 0.3:
            if pd.api.types.is_numeric_dtype(gender):
                uniq = sorted(pd.unique(gender.dropna()))
                if set(uniq).issubset({0,1}):
                    gender01 = gender.astype(int)
                elif set(uniq).issubset({1,2}):
                    gender01 = gender.map({1:1, 2:0}).astype(int)
                else:
                    raise ValueError(f"Gender column '{gcol}' could not be coerced to 0/1.")
            else:
                raise ValueError(f"Gender column '{gcol}' could not be coerced to 0/1.")
        else:
            gender01 = gender01.astype(int)

    # Label
    if label_col:
        lab = df[label_col]
        lab01 = coerce_binary(lab, positive_hint="positive").astype("float")
        if not set(pd.unique(lab01.dropna())).issubset({0.0,1.0}):
            raise ValueError(f"Label column '{label_col}' is not binary 0/1 after coercion.")
        lab01 = lab01.astype(int)
        return gender01, lab01, gcol, label_col
    else:
        label_col_inferred, lab01 = infer_binary_label(df, exclude_cols={gcol})
        return gender01, lab01, gcol, label_col_inferred

# ------------------------------
# EMD + bootstrap p-value
# ------------------------------
def compute_emd_pvalue_binary(all_labels01: np.ndarray,
                              subgroup_labels01: np.ndarray,
                              n_iter: int = 10000,
                              seed: int = 42):
    """
    Earth Mover's Distance between subgroup label distribution and overall label distribution.
    Labels must be 0/1 (binary). Permutation bootstrap p-value.
    """
    rng = np.random.default_rng(seed)
    all_labels01 = all_labels01.astype(int)
    subgroup_labels01 = subgroup_labels01.astype(int)

    observed = wasserstein_distance(subgroup_labels01, all_labels01)

    n = len(all_labels01)
    k = len(subgroup_labels01)
    boot = np.empty(n_iter, dtype=float)
    for i in range(n_iter):
        idx = rng.choice(n, size=k, replace=False)
        boot[i] = wasserstein_distance(all_labels01[idx], all_labels01)

    p_val = float((boot >= observed).mean())
    return float(observed), p_val

# ------------------------------
# Main runner
# ------------------------------
def run_for_dataset(tag: str, cfg: dict, n_iter: int = 10000):
    df = pd.read_csv(cfg["path"])
    df = df.dropna(axis=0, how="any").reset_index(drop=True)

    gender01, label01, gcol, lcol = get_gender_and_label(
        df,
        gender_col=cfg.get("gender_col"),
        label_col=cfg.get("label_col"),
        gender_map=cfg.get("gender_map"),
    )

    # Masks
    male_mask   = (gender01 == 1)
    female_mask = (gender01 == 0)

    all_labels    = label01
    female_labels = label01[female_mask]
    male_labels   = label01[male_mask]

    fem_emd, fem_p = compute_emd_pvalue_binary(all_labels, female_labels, n_iter=n_iter)
    mal_emd, mal_p = compute_emd_pvalue_binary(all_labels, male_labels,   n_iter=n_iter)

    out = pd.DataFrame([
        {"Dataset": tag, "Group": "Female", "EMD value": fem_emd, "p-value": fem_p,
         "Exhibit data bias?": "Yes" if fem_p <= 0.05 else "No"},
        {"Dataset": tag, "Group": "Male",   "EMD value": mal_emd, "p-value": mal_p,
         "Exhibit data bias?": "Yes" if mal_p <= 0.05 else "No"},
    ])
    meta = {"gender_col_used": gcol, "label_col_used": lcol, "N": len(df),
            "N_female": int(female_mask.sum()), "N_male": int(male_mask.sum())}
    return out, meta

def main(n_iter=10000):
    tables = []
    metas  = []
    for tag, cfg in DATASETS.items():
        print(f"Processing {tag} ...")
        t, m = run_for_dataset(tag, cfg, n_iter=n_iter)
        tables.append(t)
        metas.append(pd.DataFrame([{"Dataset": tag, **m}]))

    table = pd.concat(tables, ignore_index=True)
    meta  = pd.concat(metas, ignore_index=True)

    print("\n=== EMD Bias Table ===")
    print(table.to_string(index=False))
    print("\n=== Detection Metadata (for reproducibility) ===")
    print(meta.to_string(index=False))

    table.to_csv("emd_bias_table.csv", index=False)
    table.to_excel("emd_bias_table.xlsx", index=False)
    meta.to_csv("emd_bias_table_meta.csv", index=False)
    print("\nSaved: emd_bias_table.csv, emd_bias_table.xlsx, emd_bias_table_meta.csv")

if __name__ == "__main__":
    # You can lower n_iter for a quick run (e.g., 2000) and increase later (e.g., 10000/50000)
    main(n_iter=2000)

Processing D7 ...
Processing D58 ...
Processing D73 ...

=== EMD Bias Table ===
Dataset  Group  EMD value  p-value Exhibit data bias?
     D7 Female   0.005531   0.0375                Yes
     D7   Male   0.002973   0.0285                Yes
    D58 Female   0.011548   0.0000                Yes
    D58   Male   0.012620   0.0000                Yes
    D73 Female   0.006623   0.0365                Yes
    D73   Male   0.005388   0.0390                Yes

=== Detection Metadata (for reproducibility) ===
Dataset   gender_col_used                label_col_used      N  N_female  N_male
     D7            gender                        cardio  70000     24470   45530
    D58               Sex               Diabetes_binary 236378    123431  112947
    D73 sex_0male_1female hospital_outcome_1alive_0dead  19051      8546   10505

Saved: emd_bias_table.csv, emd_bias_table.xlsx, emd_bias_table_meta.csv


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
import sys
import time
import warnings

warnings.filterwarnings("ignore")

# ===========================
# Speed controls
# ===========================
FAST_MODE = True          # set True to speed up everything (folds, RF trees, etc.)
FAST_SVM  = True          # set True to replace kernel SVM with fast linear (SGD/LinearSVC)

RANDOM_STATE = 42
N_SPLITS = 10
P_THRESHOLD = 0.05

VERBOSE = True
PRINT_EVERY_FOLD = 1
PRINT_MODEL_STEPS = True

def log(msg):
    if VERBOSE:
        ts = time.strftime("%H:%M:%S")
        print(f"[{ts}] {msg}", flush=True, file=sys.stdout)

# ===========================
# CONFIG: dataset schemas
# ===========================
DATASETS = {
    "D7": {
        "path": "D7.csv",
        "gender_col": "gender",                   # 1=Male, 2=Female
        "label_col": "cardio",                    # 0/1
        "gender_map": {1: 1, 2: 0},               # Male=1, Female=0
    },
    "D58": {
        "path": "D58.csv",
        "gender_col": "Sex",                      # 0=Female, 1=Male
        "label_col": "Diabetes_binary",           # 0/1
        "gender_map": {0: 0, 1: 1},
    },
    "D73": {
        "path": "D73.csv",
        "gender_col": "sex_0male_1female",        # 0=Male, 1=Female
        "label_col": "hospital_outcome_1alive_0dead",  # 1=alive, 0=dead
        "gender_map": {0: 1, 1: 0},               # Male=1, Female=0
    },
}

# ===========================
# Models
# ===========================
def get_models():
    rf_estimators = 150 if FAST_MODE else 300
    if FAST_SVM:
        # Fast linear margin classifier standing in for SVM
        # Option 1: SGDClassifier (very fast, supports decision_function)
        svm_clf = Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", SGDClassifier(
                loss="hinge",
                alpha=1e-4,
                max_iter=2000 if FAST_MODE else 4000,
                tol=1e-3,
                random_state=RANDOM_STATE
            ))
        ])
        # Option 2: LinearSVC (also fast); uncomment to use instead of SGD
        # svm_clf = Pipeline([
        #     ("scaler", StandardScaler(with_mean=True, with_std=True)),
        #     ("clf", LinearSVC(
        #         C=1.0,
        #         max_iter=5000 if FAST_MODE else 10000,
        #         random_state=RANDOM_STATE
        #     ))
        # ])
    else:
        # Slower but classic SVM; keep probability=False to avoid Platt scaling
        svm_clf = Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", SVC(kernel="rbf", probability=False, cache_size=1000, random_state=RANDOM_STATE))
        ])

    return {
        "LR": Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", LogisticRegression(max_iter=1000, n_jobs=None, random_state=RANDOM_STATE))
        ]),
        "KNN": Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", KNeighborsClassifier(n_neighbors=5))
        ]),
        "DT": DecisionTreeClassifier(random_state=RANDOM_STATE),
        "RF": RandomForestClassifier(
            n_estimators=rf_estimators,
            max_depth=None,
            n_jobs=-1,
            random_state=RANDOM_STATE
        ),
        "SVM": svm_clf,
    }

# ===========================
# Utilities
# ===========================
def fmt_p(p):
    try:
        if p is None or np.isnan(p) or p > P_THRESHOLD:
            return "…"
        return f"{p:.4f}"
    except Exception:
        return "…"

def safe_div(a, b):
    return np.nan if (b is None or b == 0) else (a / b)

def confusion_by_group(y_true, y_pred, group_mask):
    yt = y_true[group_mask]
    yp = y_pred[group_mask]
    if yt.size == 0:
        return dict(TP=0, FP=0, TN=0, FN=0, TPR=np.nan, FPR=np.nan, FNR=np.nan, PPR=np.nan)
    TP = int(((yp == 1) & (yt == 1)).sum())
    FP = int(((yp == 1) & (yt == 0)).sum())
    TN = int(((yp == 0) & (yt == 0)).sum())
    FN = int(((yp == 0) & (yt == 1)).sum())
    TPR = safe_div(TP, TP + FN)
    FPR = safe_div(FP, FP + TN)
    FNR = safe_div(FN, TP + FN)
    PPR = safe_div((yp == 1).sum(), yp.size)
    return dict(TP=TP, FP=FP, TN=TN, FN=FN, TPR=TPR, FPR=FPR, FNR=FNR, PPR=PPR)

def score_by_group(y_score, group_mask):
    ys = y_score[group_mask]
    if ys.size == 0:
        return np.nan
    return float(np.mean(ys))

def wilcoxon_paired_zero_center(diffs):
    diffs = np.array(diffs, dtype=float)
    diffs = diffs[~np.isnan(diffs)]
    if diffs.size == 0:
        return None
    if np.allclose(diffs, 0.0):
        return 1.0
    try:
        stat, p = wilcoxon(
            diffs,
            zero_method="wilcox",
            alternative="two-sided",
            correction=False,
            mode="exact" if diffs.size <= 25 else "approx"
        )
        return float(p)
    except Exception:
        try:
            stat, p = wilcoxon(
                diffs,
                zero_method="wilcox",
                alternative="two-sided",
                correction=False,
                mode="approx"
            )
            return float(p)
        except Exception:
            return None

def prepare_xy(df, gender_col, label_col):
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    assert gender_col in df.columns and label_col in df.columns, "Gender/label column not found."
    feature_cols = [c for c in numeric_cols if c not in {gender_col, label_col}]
    X = df[feature_cols].values
    y = df[label_col].astype(int).values
    g = df[gender_col].astype(int).values  # Male=1, Female=0
    return X, y, g, feature_cols

def load_dataset(cfg):
    log(f"Loading dataset from {cfg['path']}")
    df = pd.read_csv(cfg["path"])
    log(f"Loaded shape: {df.shape}")
    if "gender_map" in cfg and cfg["gender_map"] is not None:
        log(f"Applying gender map for column '{cfg['gender_col']}' -> Male=1, Female=0")
        df[cfg["gender_col"]] = df[cfg["gender_col"]].map(cfg["gender_map"])
    lab_min, lab_max = df[cfg["label_col"]].min(), df[cfg["label_col"]].max()
    log(f"Label column '{cfg['label_col']}' value range before coercion: [{lab_min}, {lab_max}]")
    df[cfg["label_col"]] = (df[cfg["label_col"]].astype(float) > 0).astype(int)
    used = [cfg["gender_col"], cfg["label_col"]]
    before = df.shape[0]
    df = df.dropna(subset=used).copy()
    after = df.shape[0]
    log(f"Dropped {before - after} rows due to NA in gender/label; remaining: {after}")
    before = df.shape[0]
    df = df[df[cfg["gender_col"]].isin([0, 1])]
    df = df[df[cfg["label_col"]].isin([0, 1])]
    after = df.shape[0]
    if after < before:
        log(f"Removed {before - after} rows due to non-binary gender/label; remaining: {after}")
    before = df.shape[0]
    df = df.dropna(axis=0, how="any")
    after = df.shape[0]
    if after < before:
        log(f"Dropped {before - after} rows due to NA in features; remaining: {after}")
    log("Dataset ready.")
    return df

# ===========================
# Fairness evaluation (per model, per dataset)
# ===========================
def evaluate_fairness_for_models(X, y, g, models):
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    results = {name: {
        "dp_diff": [],
        "eo_tpr_diff": [],
        "eo_fpr_diff": [],
        "eopp_tpr_diff": [],
        "fpr_par_diff": [],
        "te_log_ratio_diff": [],
        "ds_score_diff": [],
        "di_log_ratio": [],
        "ber_diff": [],
    } for name in models.keys()}

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
        if (fold - 1) % PRINT_EVERY_FOLD == 0:
            log(f"Starting fold {fold}/{N_SPLITS} (train n={len(train_idx)}, test n={len(test_idx)})")

        Xtr, Xte = X[train_idx], X[test_idx]
        ytr, yte = y[train_idx], y[test_idx]
        gtr, gte = g[train_idx], g[test_idx]

        female_mask = (gte == 0)
        male_mask   = (gte == 1)

        for name, model in models.items():
            if PRINT_MODEL_STEPS:
                log(f"  [{name}] fitting on fold {fold}")
            clf = model
            clf.fit(Xtr, ytr)

            if PRINT_MODEL_STEPS:
                log(f"  [{name}] predicting on fold {fold}")
            y_pred = clf.predict(Xte)

            if hasattr(clf, "predict_proba"):
                y_score = clf.predict_proba(Xte)[:, 1]
            else:
                if hasattr(clf, "decision_function"):
                    dfun = clf.decision_function(Xte)
                    mn, mx = np.min(dfun), np.max(dfun)
                    y_score = (dfun - mn) / (mx - mn + 1e-12)
                else:
                    y_score = y_pred.astype(float)

            gf = confusion_by_group(yte, y_pred, female_mask)
            gm = confusion_by_group(yte, y_pred, male_mask)

            ppr_f, ppr_m = gf["PPR"], gm["PPR"]
            tpr_f, tpr_m = gf["TPR"], gm["TPR"]
            fpr_f, fpr_m = gf["FPR"], gm["FPR"]
            fnr_f, fnr_m = gf["FNR"], gm["FNR"]

            eopp_diff = (tpr_f - tpr_m) if (not np.isnan(tpr_f) and not np.isnan(tpr_m)) else np.nan
            ratio_f = safe_div(fnr_f, fpr_f) if (not np.isnan(fnr_f) and not np.isnan(fpr_f)) else np.nan
            ratio_m = safe_div(fnr_m, fpr_m) if (not np.isnan(fnr_m) and not np.isnan(fpr_m)) else np.nan
            te_log_ratio_diff = np.nan
            if not np.isnan(ratio_f) and not np.isnan(ratio_m) and ratio_f and ratio_m:
                te_log_ratio_diff = np.log(ratio_f + 1e-12) - np.log(ratio_m + 1e-12)

            ds_diff = score_by_group(y_score, female_mask) - score_by_group(y_score, male_mask)
            di_lr = np.nan
            if not np.isnan(ppr_f) and not np.isnan(ppr_m) and ppr_f and ppr_m:
                di_lr = np.log((ppr_f + 1e-12) / (ppr_m + 1e-12))

            ber_f = np.nanmean([fnr_f, fpr_f])
            ber_m = np.nanmean([fnr_m, fpr_m])
            ber_diff = ber_f - ber_m if (not np.isnan(ber_f) and not np.isnan(ber_m)) else np.nan

            results[name]["dp_diff"].append((ppr_f - ppr_m) if (not np.isnan(ppr_f) and not np.isnan(ppr_m)) else np.nan)
            results[name]["eo_tpr_diff"].append((tpr_f - tpr_m) if (not np.isnan(tpr_f) and not np.isnan(tpr_m)) else np.nan)
            results[name]["eo_fpr_diff"].append((fpr_f - fpr_m) if (not np.isnan(fpr_f) and not np.isnan(fpr_m)) else np.nan)
            results[name]["eopp_tpr_diff"].append(eopp_diff)
            results[name]["fpr_par_diff"].append((fpr_f - fpr_m) if (not np.isnan(fpr_f) and not np.isnan(fpr_m)) else np.nan)
            results[name]["te_log_ratio_diff"].append(te_log_ratio_diff)
            results[name]["ds_score_diff"].append(ds_diff)
            results[name]["di_log_ratio"].append(di_lr)
            results[name]["ber_diff"].append(ber_diff)

        if (fold - 1) % PRINT_EVERY_FOLD == 0:
            log(f"Completed fold {fold}/{N_SPLITS}")

    return results

def pvalues_from_results(results):
    out = {}
    for name, vals in results.items():
        log(f"Computing Wilcoxon p-values for model {name}")
        dp_p     = wilcoxon_paired_zero_center(vals["dp_diff"])
        eo_tpr_p = wilcoxon_paired_zero_center(vals["eo_tpr_diff"])
        eo_fpr_p = wilcoxon_paired_zero_center(vals["eo_fpr_diff"])
        eopp_p   = wilcoxon_paired_zero_center(vals["eopp_tpr_diff"])
        fpr_p    = wilcoxon_paired_zero_center(vals["fpr_par_diff"])
        te_p     = wilcoxon_paired_zero_center(vals["te_log_ratio_diff"])
        ds_p     = wilcoxon_paired_zero_center(vals["ds_score_diff"])
        di_p     = wilcoxon_paired_zero_center(vals["di_log_ratio"])
        ber_p    = wilcoxon_paired_zero_center(vals["ber_diff"])

        out[name] = {
            "Demographic parity": fmt_p(dp_p),
            "Equalised odds": f"TPR={fmt_p(eo_tpr_p)}, FPR={fmt_p(eo_fpr_p)}",
            "Equal opportunity": fmt_p(eopp_p),
            "False positive rate parity": fmt_p(fpr_p),
            "Treatment equality": fmt_p(te_p),
            "Discrimination score": fmt_p(ds_p),
            "Disparate Impact": fmt_p(di_p),
            "Balanced Error Rate": fmt_p(ber_p),
        }
    return out

def build_table(pvalues_map):
    fairness_rows = [
        "Demographic parity",
        "Equalised odds",
        "Equal opportunity",
        "False positive rate parity",
        "Treatment equality",
        "Discrimination score",
        "Disparate Impact",
        "Balanced Error Rate",
    ]
    model_names = list(pvalues_map.keys())
    data = {m: [pvalues_map[m][row] for row in fairness_rows] for m in model_names}
    return pd.DataFrame(data, index=fairness_rows)

# ===========================
# Orchestrator
# ===========================
def run_for_dataset(tag, cfg):
    log(f"=== Processing {tag} ===")
    df = load_dataset(cfg)
    log("Preparing X, y, g matrices")
    X, y, g, feature_cols = prepare_xy(df, cfg["gender_col"], cfg["label_col"])
    log(f"Feature matrix shape: {X.shape}, labels shape: {y.shape}, gender shape: {g.shape}")
    models = get_models()
    log(f"Evaluating fairness across {len(models)} models with {N_SPLITS}-fold CV (FAST_MODE={FAST_MODE}, FAST_SVM={FAST_SVM})")
    results = evaluate_fairness_for_models(X, y, g, models)
    log("Aggregating per-fold differences into Wilcoxon tests")
    pvals = pvalues_from_results(results)
    table = build_table(pvals)
    log("Final table for this dataset:")
    log("\n" + table.to_string())
    out_csv = f"fairness_table_{tag}.csv"
    out_xlsx = f"fairness_table_{tag}.xlsx"
    table.to_csv(out_csv)
    log(f"Saved {out_csv}")
    try:
        table.to_excel(out_xlsx)
        log(f"Saved {out_xlsx}")
    except Exception as e:
        log(f"Could not save Excel for {tag}: {e}")
    log(f"=== Done {tag} ===")
    return table

def main():
    log("Starting fairness evaluation for all datasets")
    tables = {}
    for i, (tag, cfg) in enumerate(DATASETS.items(), start=1):
        log(f"Dataset {i}/{len(DATASETS)}: {tag}")
        tables[tag] = run_for_dataset(tag, cfg)
    try:
        log("Writing combined multi-sheet Excel: fairness_tables_all.xlsx")
        with pd.ExcelWriter("fairness_tables_all.xlsx", engine="xlsxwriter") as writer:
            for tag, tbl in tables.items():
                tbl.to_excel(writer, sheet_name=tag)
        log("Saved fairness_tables_all.xlsx")
    except Exception as e:
        log(f"Could not write combined Excel: {e}")
    log("All done.")

if __name__ == "__main__":
    main()

[07:30:59] Starting fairness evaluation for all datasets
[07:30:59] Dataset 1/3: D7
[07:30:59] === Processing D7 ===
[07:30:59] Loading dataset from D7.csv
[07:30:59] Loaded shape: (70000, 14)
[07:30:59] Applying gender map for column 'gender' -> Male=1, Female=0
[07:30:59] Label column 'cardio' value range before coercion: [0, 1]
[07:30:59] Dropped 0 rows due to NA in gender/label; remaining: 70000
[07:30:59] Dataset ready.
[07:30:59] Preparing X, y, g matrices
[07:30:59] Feature matrix shape: (70000, 12), labels shape: (70000,), gender shape: (70000,)
[07:30:59] Evaluating fairness across 5 models with 10-fold CV (FAST_MODE=True, FAST_SVM=True)
[07:30:59] Starting fold 1/10 (train n=63000, test n=7000)
[07:30:59]   [LR] fitting on fold 1
[07:31:00]   [LR] predicting on fold 1
[07:31:00]   [KNN] fitting on fold 1
[07:31:00]   [KNN] predicting on fold 1
[07:31:11]   [DT] fitting on fold 1
[07:31:12]   [DT] predicting on fold 1
[07:31:12]   [RF] fitting on fold 1
[07:31:32]   [RF] predi

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance
import warnings
warnings.filterwarnings("ignore")

# ==========================
# Configuration
# ==========================
DATASETS = {
    # D7: remove 81 males from class 0
    "D7": {
        "path": "D7.csv",
        "gender_col": "gender",                 # 1=Male, 2=Female
        "label_col": "cardio",                  # 0/1
        "gender_map": {1: 1, 2: 0},             # Male=1, Female=0
        "removal": {"gender": 1, "label": 0, "n": 162}, #81
    },
    # D58: remove 17,213 females from class 0
    "D58": {
        "path": "D58.csv",
        "gender_col": "Sex",                    # 0=Female, 1=Male
        "label_col": "Diabetes_binary",         # 0/1
        "gender_map": {0: 0, 1: 1},             # Male=1, Female=0
        "removal": {"gender": 0, "label": 0, "n": 34426}, #17213
    },
    # D73: remove 460 males from class 0 (0=dead)
    "D73": {
        "path": "D73.csv",
        "gender_col": "sex_0male_1female",      # 0=Male, 1=Female
        "label_col": "hospital_outcome_1alive_0dead",  # 1=alive, 0=dead
        "gender_map": {0: 1, 1: 0},             # Male=1, Female=0
        "removal": {"gender": 1, "label": 0, "n": 920},#460
    },
}

REMOVAL_DESCRIPTIONS = {
    "D7":  "81, Male from 0 class",
    "D58": "17,213, female from 0 class",
    "D73": "460, male from 0 class",
}

# ==========================
# Helpers
# ==========================
def fmt_p(p, threshold=0.05):
    try:
        if p is None or np.isnan(p) or p > threshold:
            return "…"
        return f"{p:.4f}"
    except Exception:
        return "…"

def compute_emd_pvalue_binary(all_labels01: np.ndarray,
                              subgroup_labels01: np.ndarray,
                              n_iter: int = 2000,
                              seed: int = 42):
    """
    EMD between subgroup label distribution and overall label distribution (binary labels).
    Permutation bootstrap p-value using sampling without replacement.
    """
    rng = np.random.default_rng(seed)
    all_labels01 = np.asarray(all_labels01, dtype=int)
    subgroup_labels01 = np.asarray(subgroup_labels01, dtype=int)

    observed = wasserstein_distance(subgroup_labels01, all_labels01)

    n = len(all_labels01)
    k = len(subgroup_labels01)
    boot = np.empty(n_iter, dtype=float)
    for i in range(n_iter):
        idx = rng.choice(n, size=k, replace=False)
        boot[i] = wasserstein_distance(all_labels01[idx], all_labels01)

    p_val = float((boot >= observed).mean())
    return float(observed), p_val

def load_and_prepare(cfg):
    df = pd.read_csv(cfg["path"])
    # Map gender -> {Male:1, Female:0}
    if "gender_map" in cfg and cfg["gender_map"] is not None:
        df[cfg["gender_col"]] = df[cfg["gender_col"]].map(cfg["gender_map"])
    # Coerce label to 0/1
    df[cfg["label_col"]] = (df[cfg["label_col"]].astype(float) > 0).astype(int)
    # Keep only valid rows
    df = df.dropna(subset=[cfg["gender_col"], cfg["label_col"]]).copy()
    df = df[df[cfg["gender_col"]].isin([0, 1])]
    df = df[df[cfg["label_col"]].isin([0, 1])]
    df = df.dropna(axis=0, how="any")
    return df

def remove_instances(df, gender_col, label_col, gender_val, label_val, n_remove, seed=42):
    """
    Remove n_remove rows uniformly at random where (gender==gender_val & label==label_val).
    If fewer rows exist, remove all available.
    """
    mask = (df[gender_col] == gender_val) & (df[label_col] == label_val)
    idx = df[mask].index
    if len(idx) == 0:
        return df
    n = min(n_remove, len(idx))
    rng = np.random.default_rng(seed)
    drop_idx = rng.choice(idx, size=n, replace=False)
    return df.drop(index=drop_idx)

def emd_table_for_dataset(tag, cfg, n_iter=2000, seed=42):
    df = load_and_prepare(cfg)

    # Apply specified removal
    rem = cfg.get("removal", None)
    if rem is not None:
        df = remove_instances(
            df,
            gender_col=cfg["gender_col"],
            label_col=cfg["label_col"],
            gender_val=rem["gender"],
            label_val=rem["label"],
            n_remove=rem["n"],
            seed=seed,
        )

    # Build arrays
    y = df[cfg["label_col"]].astype(int).values
    g = df[cfg["gender_col"]].astype(int).values  # Male=1, Female=0

    female_mask = (g == 0)
    male_mask   = (g == 1)

    all_labels    = y
    female_labels = y[female_mask]
    male_labels   = y[male_mask]

    fem_emd, fem_p = compute_emd_pvalue_binary(all_labels, female_labels, n_iter=n_iter, seed=seed)
    mal_emd, mal_p = compute_emd_pvalue_binary(all_labels, male_labels,   n_iter=n_iter, seed=seed)

    out = pd.DataFrame([
        {"Dataset": tag, "Removal of data instances": REMOVAL_DESCRIPTIONS.get(tag, ""), "Group": "Female",
         "EMD value": round(fem_emd, 6), "p-value": fmt_p(fem_p),
         "Exhibit data bias?": "Yes" if (isinstance(fem_p, float) and fem_p <= 0.05) else "No"},
        {"Dataset": tag, "Removal of data instances": "", "Group": "Male",
         "EMD value": round(mal_emd, 6), "p-value": fmt_p(mal_p),
         "Exhibit data bias?": "Yes" if (isinstance(mal_p, float) and mal_p <= 0.05) else "No"},
    ])
    return out

# ==========================
# Main
# ==========================
def main():
    tables = []
    for tag, cfg in DATASETS.items():
        t = emd_table_for_dataset(tag, cfg, n_iter=2000, seed=42)
        tables.append(t)
    table = pd.concat(tables, ignore_index=True)

    # Pretty print
    pd.set_option("display.max_colwidth", None)
    print("\n=== EMD Bias Table after Specified Instance Removal ===")
    print(table.to_string(index=False))

    # Save
    table.to_csv("emd_bias_table_after_specified_removal.csv", index=False)
    try:
        table.to_excel("emd_bias_table_after_specified_removal.xlsx", index=False)
    except Exception:
        pass

if __name__ == "__main__":
    main()


=== EMD Bias Table after Specified Instance Removal ===
Dataset   Removal of data instances  Group  EMD value p-value Exhibit data bias?
     D7       81, Male from 0 class Female   0.004372       …                 No
     D7                               Male   0.002358       …                 No
    D58 17,213, female from 0 class Female   0.014705  0.0000                Yes
    D58                               Male   0.011588  0.0000                Yes
    D73      460, male from 0 class Female   0.034514  0.0000                Yes
    D73                               Male   0.030773  0.0000                Yes


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

# ==========================
# Dataset configs
# ==========================
DATASETS = {
    "D7": {
        "path": "D7.csv",
        "gender_col": "gender",
        "label_col": "cardio",
        "gender_map": {1: 1, 2: 0},  # Male=1, Female=0
        "removal": {"gender": 1, "label": 0, "n": 81}
    },
    "D58": {
        "path": "D58.csv",
        "gender_col": "Sex",
        "label_col": "Diabetes_binary",
        "gender_map": {0: 0, 1: 1},  # Female=0, Male=1
        "removal": {"gender": 0, "label": 0, "n": 17213}
    },
    "D73": {
        "path": "D73.csv",
        "gender_col": "sex_0male_1female",
        "label_col": "hospital_outcome_1alive_0dead",
        "gender_map": {0: 1, 1: 0},  # Male=1, Female=0
        "removal": {"gender": 1, "label": 0, "n": 460}
    }
}

# ==========================
# Models
# ==========================
MODELS = {
    "LR": LogisticRegression(max_iter=2000),
    "KNN": KNeighborsClassifier(),
    "DT": DecisionTreeClassifier(random_state=42),
    "RF": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=False)  # may be slower
}

# ==========================
# Helpers
# ==========================
def load_and_prepare(cfg):
    df = pd.read_csv(cfg["path"])
    df[cfg["gender_col"]] = df[cfg["gender_col"]].map(cfg["gender_map"])
    df[cfg["label_col"]] = (df[cfg["label_col"]].astype(float) > 0).astype(int)
    df = df.dropna(subset=[cfg["gender_col"], cfg["label_col"]])
    return df

def remove_instances(df, cfg, seed=42):
    rem = cfg.get("removal", None)
    if rem is None:
        return df
    mask = (df[cfg["gender_col"]] == rem["gender"]) & (df[cfg["label_col"]] == rem["label"])
    idx = df[mask].index
    if len(idx) == 0:
        return df
    n = min(rem["n"], len(idx))
    rng = np.random.default_rng(seed)
    drop_idx = rng.choice(idx, size=n, replace=False)
    return df.drop(index=drop_idx)

def evaluate_models(X_train, X_test, y_train, y_test):
    results = {}
    for name, model in MODELS.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_train)  # training phase results
        results[name] = {
            "Accuracy": accuracy_score(y_train, y_pred),
            "Precision": precision_score(y_train, y_pred, zero_division=0),
            "Recall": recall_score(y_train, y_pred, zero_division=0),
            "F1-score": f1_score(y_train, y_pred, zero_division=0)
        }
    return results

# ==========================
# Main
# ==========================
def main():
    all_tables = {}
    for tag, cfg in DATASETS.items():
        df_orig = load_and_prepare(cfg)

        # features: drop gender & label (keep all else as predictors)
        X = df_orig.drop(columns=[cfg["label_col"], cfg["gender_col"]])
        y = df_orig[cfg["label_col"]]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        results_before = evaluate_models(X_train, X_test, y_train, y_test)

        # After removal
        df_after = remove_instances(df_orig.copy(), cfg)
        X2 = df_after.drop(columns=[cfg["label_col"], cfg["gender_col"]])
        y2 = df_after[cfg["label_col"]]

        X2_train, X2_test, y2_train, y2_test = train_test_split(
            X2, y2, test_size=0.2, random_state=42, stratify=y2
        )

        results_after = evaluate_models(X2_train, X2_test, y2_train, y2_test)

        # Build comparison table
        rows = []
        for model in MODELS.keys():
            rows.append({
                "Model": model,
                "Acc_Before": round(results_before[model]["Accuracy"], 4),
                "Prec_Before": round(results_before[model]["Precision"], 4),
                "Rec_Before": round(results_before[model]["Recall"], 4),
                "F1_Before": round(results_before[model]["F1-score"], 4),
                "Acc_After": round(results_after[model]["Accuracy"], 4),
                "Prec_After": round(results_after[model]["Precision"], 4),
                "Rec_After": round(results_after[model]["Recall"], 4),
                "F1_After": round(results_after[model]["F1-score"], 4),
            })
        all_tables[tag] = pd.DataFrame(rows)

        print(f"\n=== {tag} Results ===")
        print(all_tables[tag].to_string(index=False))

        all_tables[tag].to_csv(f"{tag}_before_after_metrics.csv", index=False)

    return all_tables

if __name__ == "__main__":
    tables = main()


=== D7 Results ===
Model  Acc_Before  Prec_Before  Rec_Before  F1_Before  Acc_After  Prec_After  Rec_After  F1_After
   LR      0.7038       0.7208      0.6647     0.6916     0.7074      0.7260     0.6668    0.6951
  KNN      0.7145       0.7160      0.7103     0.7132     0.7155      0.7174     0.7116    0.7145
   DT      1.0000       1.0000      1.0000     1.0000     1.0000      1.0000     1.0000    1.0000
   RF      1.0000       1.0000      1.0000     1.0000     1.0000      1.0000     1.0000    1.0000
  SVM      0.5987       0.5874      0.6619     0.6224     0.5981      0.5863     0.6680    0.6245


In [ ]:
import pandas as pd
import numpy as np
import time
import sys
import warnings

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC  # fast linear SVM (kept as SVM per requirement)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

warnings.filterwarnings("ignore")

# ==========================
# Progress / logging
# ==========================
VERBOSE = True
def log(msg):
    if VERBOSE:
        ts = time.strftime("%H:%M:%S")
        print(f"[{ts}] {msg}", file=sys.stdout, flush=True)

# ==========================
# Dataset configs (80:20 split; results on TRAINING predictions)
# ==========================
DATASETS = {
    "D58": {
        "path": "D58.csv",
        "gender_col": "Sex",                           # 0=Female, 1=Male
        "label_col": "Diabetes_binary",                # 0/1
        "gender_map": {0: 0, 1: 1},                    # Female=0, Male=1
        "removal": {"gender": 0, "label": 0, "n": 17213}, # remove 17,213 females from class 0
    },
    "D73": {
        "path": "D73.csv",
        "gender_col": "sex_0male_1female",             # 0=Male, 1=Female
        "label_col": "hospital_outcome_1alive_0dead",  # 1=alive, 0=dead
        "gender_map": {0: 1, 1: 0},                    # Male=1, Female=0
        "removal": {"gender": 1, "label": 0, "n": 460},   # remove 460 males from class 0 (dead)
    },
}

RANDOM_STATE = 42
TEST_SIZE = 0.2

# ==========================
# Models (keep SVM; use LinearSVC for speed)
# ==========================
def get_models():
    models = {
        "LR": Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", LogisticRegression(max_iter=2000, random_state=RANDOM_STATE))
        ]),
        "KNN": Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", KNeighborsClassifier(n_neighbors=5))
        ]),
        "DT": DecisionTreeClassifier(random_state=RANDOM_STATE),
        "RF": RandomForestClassifier(
            n_estimators=200,        # tuned down for speed
            max_depth=None,
            n_jobs=-1,
            random_state=RANDOM_STATE
        ),
        # Keep SVM: LinearSVC (fast). Still an SVM (hinge loss, linear kernel).
        "SVM": Pipeline([
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("clf", LinearSVC(C=1.0, max_iter=5000, random_state=RANDOM_STATE))
        ]),
    }
    return models

# ==========================
# Helpers
# ==========================
def load_and_prepare(cfg):
    log(f"Loading: {cfg['path']}")
    df = pd.read_csv(cfg["path"])
    log(f"Loaded shape: {df.shape}")

    # Map gender -> {Male:1, Female:0}
    if "gender_map" in cfg and cfg["gender_map"] is not None:
        before_unique = df[cfg["gender_col"]].unique()
        df[cfg["gender_col"]] = df[cfg["gender_col"]].map(cfg["gender_map"])
        after_unique = df[cfg["gender_col"]].unique()
        log(f"Mapped gender '{cfg['gender_col']}' values {before_unique} -> {after_unique}")

    # Coerce label to 0/1 (strict)
    before_label_unique = df[cfg["label_col"]].unique()
    df[cfg["label_col"]] = (df[cfg["label_col"]].astype(float) > 0).astype(int)
    after_label_unique = df[cfg["label_col"]].unique()
    log(f"Coerced label '{cfg['label_col']}' {before_label_unique} -> {after_label_unique}")

    # Keep valid rows
    used_cols = [cfg["gender_col"], cfg["label_col"]]
    before = len(df)
    df = df.dropna(subset=used_cols).copy()
    df = df[df[cfg["gender_col"]].isin([0, 1])]
    df = df[df[cfg["label_col"]].isin([0, 1])]
    after = len(df)
    log(f"Filtered invalid rows: {before - after} removed, remaining: {after}")

    # Drop remaining NA in features
    before = len(df)
    df = df.dropna(axis=0, how="any")
    after = len(df)
    if after < before:
        log(f"Dropped rows with NA in features: {before - after}, remaining: {after}")

    return df

def remove_instances(df, cfg, seed=RANDOM_STATE):
    rem = cfg.get("removal", None)
    if rem is None:
        return df, 0
    g_col, y_col = cfg["gender_col"], cfg["label_col"]
    mask = (df[g_col] == rem["gender"]) & (df[y_col] == rem["label"])
    idx = df[mask].index
    n_avail = len(idx)
    n = min(rem["n"], n_avail)
    if n == 0:
        log("No matching instances to remove.")
        return df, 0
    rng = np.random.default_rng(seed)
    drop_idx = rng.choice(idx, size=n, replace=False)
    df2 = df.drop(index=drop_idx)
    log(f"Removed {n} rows (requested {rem['n']}, available {n_avail}) "
        f"where gender={rem['gender']}, label={rem['label']}. New shape: {df2.shape}")
    return df2, n

def split_xy(df, cfg):
    # Features: all numeric columns except gender & label
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feat_cols = [c for c in numeric_cols if c not in {cfg["gender_col"], cfg["label_col"]}]
    X = df[feat_cols].values
    y = df[cfg["label_col"]].values
    return X, y, feat_cols

def evaluate_on_training(X_train, y_train):
    models = get_models()
    results = {}
    for name, model in models.items():
        t0 = time.time()
        log(f"  [{name}] fit on training set: n={X_train.shape[0]}, d={X_train.shape[1]}")
        model.fit(X_train, y_train)
        t1 = time.time()
        y_pred = model.predict(X_train)
        acc = accuracy_score(y_train, y_pred)
        prec = precision_score(y_train, y_pred, zero_division=0)
        rec = recall_score(y_train, y_pred, zero_division=0)
        f1 = f1_score(y_train, y_pred, zero_division=0)
        dt = t1 - t0
        log(f"  [{name}] done. Train Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f} (fit {dt:.2f}s)")
        results[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-score": f1, "fit_seconds": dt}
    return results

def format_table(results_before, results_after):
    rows = []
    for name in ["LR", "KNN", "DT", "RF", "SVM"]:
        b = results_before[name]
        a = results_after[name]
        rows.append({
            "Model": name,
            "Accuracy (Before)": round(b["Accuracy"], 4),
            "Precision (Before)": round(b["Precision"], 4),
            "Recall (Before)": round(b["Recall"], 4),
            "F1-score (Before)": round(b["F1-score"], 4),
            "Accuracy (After)": round(a["Accuracy"], 4),
            "Precision (After)": round(a["Precision"], 4),
            "Recall (After)": round(a["Recall"], 4),
            "F1-score (After)": round(a["F1-score"], 4),
        })
    cols = [
        "Model",
        "Accuracy (Before)", "Precision (Before)", "Recall (Before)", "F1-score (Before)",
        "Accuracy (After)",  "Precision (After)",  "Recall (After)",  "F1-score (After)"
    ]
    return pd.DataFrame(rows, columns=cols)

# ==========================
# Orchestrator
# ==========================
def run_dataset(tag, cfg):
    log(f"\n=== [{tag}] START ===")
    df = load_and_prepare(cfg)

    # 80:20 split (original dataset)
    X, y, feat_cols = split_xy(df, cfg)
    log(f"[{tag}] Features: {len(feat_cols)}; Splitting 80:20 (original)")
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
    )
    log(f"[{tag}] Train n={X_tr.shape[0]}, Test n={X_te.shape[0]} (original)")
    log(f"[{tag}] Training models on ORIGINAL training set (printing training metrics)")
    res_before = evaluate_on_training(X_tr, y_tr)

    # Apply removal, then split again and train
    log(f"[{tag}] Applying instance removal")
    df_after, n_removed = remove_instances(df, cfg)
    X2, y2, feat_cols2 = split_xy(df_after, cfg)
    log(f"[{tag}] Features after removal: {len(feat_cols2)}; Splitting 80:20 (after removal)")
    X2_tr, X2_te, y2_tr, y2_te = train_test_split(
        X2, y2, test_size=TEST_SIZE, stratify=y2, random_state=RANDOM_STATE
    )
    log(f"[{tag}] Train n={X2_tr.shape[0]}, Test n={X2_te.shape[0]} (after removal)")
    log(f"[{tag}] Training models on AFTER-REMOVAL training set (printing training metrics)")
    res_after = evaluate_on_training(X2_tr, y2_tr)

    table = format_table(res_before, res_after)
    log(f"[{tag}] Table complete:")
    log("\n" + table.to_string(index=False))

    # Save per-dataset CSV/Excel
    out_csv = f"{tag}_before_after_training_metrics.csv"
    table.to_csv(out_csv, index=False)
    try:
        table.to_excel(f"{tag}_before_after_training_metrics.xlsx", index=False)
    except Exception:
        pass

    log(f"=== [{tag}] DONE ===")
    return table

def main():
    log("Starting before/after training metrics (80:20 split, training predictions)")
    all_tables = {}
    for tag, cfg in DATASETS.items():
        all_tables[tag] = run_dataset(tag, cfg)

    # Optional: combined workbook
    try:
        log("Writing combined workbook: before_after_training_metrics_all.xlsx")
        with pd.ExcelWriter("before_after_training_metrics_all.xlsx", engine="xlsxwriter") as writer:
            for tag, tbl in all_tables.items():
                tbl.to_excel(writer, sheet_name=tag, index=False)
        log("Saved before_after_training_metrics_all.xlsx")
    except Exception as e:
        log(f"Could not write combined Excel: {e}")

    log("All datasets complete.")
    return all_tables

if __name__ == "__main__":
    main()

[09:29:42] Starting before/after training metrics (80:20 split, training predictions)
[09:29:42] 
=== [D58] START ===
[09:29:42] Loading: D58.csv
[09:29:43] Loaded shape: (236378, 22)
[09:29:43] Mapped gender 'Sex' values [0 1] -> [0 1]
[09:29:43] Coerced label 'Diabetes_binary' [0. 1.] -> [0 1]
[09:29:43] Filtered invalid rows: 0 removed, remaining: 236378
[09:29:43] [D58] Features: 20; Splitting 80:20 (original)
[09:29:44] [D58] Train n=189102, Test n=47276 (original)
[09:29:44] [D58] Training models on ORIGINAL training set (printing training metrics)
[09:29:44]   [LR] fit on training set: n=189102, d=20
[09:29:44]   [LR] done. Train Acc=0.8607, Prec=0.5346, Rec=0.1458, F1=0.2290 (fit 0.56s)
[09:29:44]   [KNN] fit on training set: n=189102, d=20
[09:33:11]   [KNN] done. Train Acc=0.8830, Prec=0.6702, Rec=0.3470, F1=0.4573 (fit 0.10s)
[09:33:11]   [DT] fit on training set: n=189102, d=20
[09:33:13]   [DT] done. Train Acc=0.9932, Prec=0.9986, Rec=0.9532, F1=0.9753 (fit 1.87s)
[09:33:1

In [ ]:
# D7 — Fairness-aware evaluation using Fairlearn (Demographic Parity) with verbose progress & timings
# FIX: ThresholdOptimizer.predict requires sensitive_features at predict-time.
# Models: LR, KNN, DT, RF, SVM(LinearSVC). Baseline is NOT trained here.

import pandas as pd
import numpy as np
import time
import sys
import warnings
from scipy.stats import wilcoxon

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

warnings.filterwarnings("ignore")

from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.postprocessing import ThresholdOptimizer

# ===========================
# Config
# ===========================
RANDOM_STATE = 42
N_SPLITS = 10
P_THRESHOLD = 0.05
VERBOSE = True

D7_CFG = {
    "path": "D7.csv",
    "gender_col": "gender",       # 1=Male, 2=Female
    "label_col": "cardio",        # 0/1
    "gender_map": {1: 1, 2: 0},   # Male=1, Female=0
}

def log(msg):
    if VERBOSE:
        ts = time.strftime("%H:%M:%S")
        print(f"[{ts}] {msg}", file=sys.stdout, flush=True)

# ===========================
# Helpers
# ===========================
def fmt_p(p):
    try:
        if p is None or np.isnan(p) or p > P_THRESHOLD:
            return "…"
        return f"{p:.4f}"
    except Exception:
        return "…"

def safe_div(a, b):
    return np.nan if (b is None or b == 0) else (a / b)

def confusion_by_group(y_true, y_pred, group_mask):
    yt = y_true[group_mask]
    yp = y_pred[group_mask]
    if yt.size == 0:
        return dict(TP=0, FP=0, TN=0, FN=0, TPR=np.nan, FPR=np.nan, FNR=np.nan, PPR=np.nan)
    TP = int(((yp == 1) & (yt == 1)).sum())
    FP = int(((yp == 1) & (yt == 0)).sum())
    TN = int(((yp == 0) & (yt == 0)).sum())
    FN = int(((yp == 0) & (yt == 1)).sum())
    TPR = safe_div(TP, TP + FN)
    FPR = safe_div(FP, FP + TN)
    FNR = safe_div(FN, TP + FN)
    PPR = safe_div((yp == 1).sum(), yp.size)
    return dict(TP=TP, FP=FP, TN=TN, FN=FN, TPR=TPR, FPR=FPR, FNR=FNR, PPR=PPR)

def score_by_group(y_score, group_mask):
    ys = y_score[group_mask]
    if ys.size == 0:
        return np.nan
    return float(np.mean(ys))

def wilcoxon_paired_zero_center(diffs):
    diffs = np.array(diffs, dtype=float)
    diffs = diffs[~np.isnan(diffs)]
    if diffs.size == 0:
        return None
    if np.allclose(diffs, 0.0):
        return 1.0
    try:
        stat, p = wilcoxon(
            diffs, zero_method="wilcox", alternative="two-sided",
            correction=False, mode="exact" if diffs.size <= 25 else "approx"
        )
        return float(p)
    except Exception:
        try:
            stat, p = wilcoxon(
                diffs, zero_method="wilcox", alternative="two-sided",
                correction=False, mode="approx"
            )
            return float(p)
        except Exception:
            return None

def load_d7(cfg):
    log("📥 Loading D7 …")
    t0 = time.time()
    df = pd.read_csv(cfg["path"])
    log(f"✅ Loaded shape: {df.shape} (took {time.time()-t0:.2f}s)")
    before = df[cfg["gender_col"]].unique()
    df[cfg["gender_col"]] = df[cfg["gender_col"]].map(cfg["gender_map"])
    after = df[cfg["gender_col"]].unique()
    log(f"↪️  Mapped gender {before} -> {after}")
    df[cfg["label_col"]] = (df[cfg["label_col"]].astype(float) > 0).astype(int)
    df = df.dropna(subset=[cfg["gender_col"], cfg["label_col"]]).copy()
    df = df[df[cfg["gender_col"]].isin([0, 1])]
    df = df[df[cfg["label_col"]].isin([0, 1])]
    df = df.dropna(axis=0, how="any")
    log(f"🧹 Cleaned shape: {df.shape}")
    return df

def split_X_y_g(df, gender_col, label_col):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feat_cols = [c for c in num_cols if c not in {gender_col, label_col}]
    X = df[feat_cols].values
    y = df[label_col].values.astype(int)
    g = df[gender_col].values.astype(int)
    return X, y, g, feat_cols

# ===========================
# Base models and mitigators
# ===========================
def base_models():
    return {
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "LR": LogisticRegression(max_iter=2000, random_state=RANDOM_STATE),
        "DT": DecisionTreeClassifier(random_state=RANDOM_STATE),
        "RF": RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=RANDOM_STATE),
        "SVM": LinearSVC(C=1.0, max_iter=5000, random_state=RANDOM_STATE)
    }

def fairness_wrappers():
    """
    - For LR/DT/RF/SVM: ExponentiatedGradient(DemographicParity) (in-processing)
    - For KNN: ThresholdOptimizer (post-processing, since KNN lacks sample_weight)
    """
    def eg_wrap(est):
        return ExponentiatedGradient(
            estimator=est,
            constraints=DemographicParity(),
            max_iter=50,
            nu=1e-6
        )
    def to_wrap(prefit_est):
        return ThresholdOptimizer(
            estimator=prefit_est,
            constraints="demographic_parity",
            predict_method="auto",
            prefit=True
        )
    return eg_wrap, to_wrap

# ===========================
# Fairness-aware CV and Wilcoxon p-values (with progress)
# ===========================
def evaluate_fairlearn_dp_verbose(X, y, g):
    log("🔹 Starting fairness-aware evaluation (Demographic Parity)")
    scaler = StandardScaler()
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    models = base_models()
    eg_wrap, to_wrap = fairness_wrappers()

    collectors = {name: {
        "dp_diff": [], "eo_tpr_diff": [], "eo_fpr_diff": [],
        "eopp_tpr_diff": [], "fpr_par_diff": [],
        "te_log_ratio_diff": [], "ds_score_diff": [],
        "di_log_ratio": [], "ber_diff": []
    } for name in models.keys()}

    for fold, (tr, te) in enumerate(skf.split(X, y), start=1):
        log(f"\n👉 Fold {fold}/{N_SPLITS} — preparing data")
        t_fold = time.time()
        Xtr, Xte = X[tr], X[te]
        ytr, yte = y[tr], y[te]
        gtr, gte = g[tr], g[te]

        t0 = time.time()
        Xtr = scaler.fit_transform(Xtr)
        Xte = scaler.transform(Xte)
        log(f"   ✨ Scaled features (took {time.time()-t0:.2f}s)")

        female_mask = (gte == 0)
        male_mask   = (gte == 1)

        for name, base in models.items():
            log(f"   ⏳ Training {name} with fairness constraints …")
            t_train = time.time()

            if name == "KNN":
                # Post-processing DP for KNN (requires sensitive_features at fit & predict)
                base_knn = KNeighborsClassifier(n_neighbors=base.n_neighbors)
                base_knn.fit(Xtr, ytr)
                mitigator = to_wrap(base_knn)
                mitigator.fit(Xtr, ytr, sensitive_features=gtr)
            else:
                # In-processing DP via ExponentiatedGradient
                base_est = base.__class__(**base.get_params())
                mitigator = eg_wrap(base_est)
                mitigator.fit(Xtr, ytr, sensitive_features=gtr)

            log(f"   ✅ {name} fitted in {time.time()-t_train:.2f}s")

            t_pred = time.time()
            if name == "KNN":
                # IMPORTANT: ThresholdOptimizer.predict REQUIRES sensitive_features
                y_pred = mitigator.predict(Xte, sensitive_features=gte)
            else:
                y_pred = mitigator.predict(Xte)
            log(f"   🔍 {name} predictions in {time.time()-t_pred:.2f}s")

            # Scores for discrimination score
            t_score = time.time()
            if hasattr(mitigator, "predict_proba"):
                try:
                    y_score = mitigator.predict_proba(Xte)[:, 1]
                except Exception:
                    y_score = y_pred.astype(float)
            else:
                if hasattr(mitigator, "decision_function"):
                    dfun = mitigator.decision_function(Xte)
                    mn, mx = np.min(dfun), np.max(dfun)
                    y_score = (dfun - mn) / (mx - mn + 1e-12)
                else:
                    if name == "KNN" and hasattr(mitigator.estimator_, "predict_proba"):
                        y_score = mitigator.estimator_.predict_proba(Xte)[:, 1]
                    else:
                        y_score = y_pred.astype(float)
            log(f"   🧮 {name} scoring prepared in {time.time()-t_score:.2f}s")

            gf = confusion_by_group(yte, y_pred, female_mask)
            gm = confusion_by_group(yte, y_pred, male_mask)

            ppr_f, ppr_m = gf["PPR"], gm["PPR"]
            tpr_f, tpr_m = gf["TPR"], gm["TPR"]
            fpr_f, fpr_m = gf["FPR"], gm["FPR"]
            fnr_f, fnr_m = gf["FNR"], gm["FNR"]

            eopp_diff = (tpr_f - tpr_m) if (not np.isnan(tpr_f) and not np.isnan(tpr_m)) else np.nan
            ratio_f = safe_div(fnr_f, fpr_f) if (not np.isnan(fnr_f) and not np.isnan(fpr_f)) else np.nan
            ratio_m = safe_div(fnr_m, fpr_m) if (not np.isnan(fnr_m) and not np.isnan(fpr_m)) else np.nan
            te_log_ratio_diff = np.nan
            if not np.isnan(ratio_f) and not np.isnan(ratio_m) and ratio_f and ratio_m:
                te_log_ratio_diff = np.log(ratio_f + 1e-12) - np.log(ratio_m + 1e-12)

            ds_diff = score_by_group(y_score, female_mask) - score_by_group(y_score, male_mask)
            di_lr = np.nan
            if not np.isnan(ppr_f) and not np.isnan(ppr_m) and ppr_f and ppr_m:
                di_lr = np.log((ppr_f + 1e-12) / (ppr_m + 1e-12))

            ber_f = np.nanmean([fnr_f, fpr_f])
            ber_m = np.nanmean([fnr_m, fpr_m])
            ber_diff = ber_f - ber_m if (not np.isnan(ber_f) and not np.isnan(ber_m)) else np.nan

            collectors[name]["dp_diff"].append((ppr_f - ppr_m) if (not np.isnan(ppr_f) and not np.isnan(ppr_m)) else np.nan)
            collectors[name]["eo_tpr_diff"].append((tpr_f - tpr_m) if (not np.isnan(tpr_f) and not np.isnan(tpr_m)) else np.nan)
            collectors[name]["eo_fpr_diff"].append((fpr_f - fpr_m) if (not np.isnan(fpr_f) and not np.isnan(fpr_m)) else np.nan)
            collectors[name]["eopp_tpr_diff"].append(eopp_diff)
            collectors[name]["fpr_par_diff"].append((fpr_f - fpr_m) if (not np.isnan(fpr_f) and not np.isnan(fpr_m)) else np.nan)
            collectors[name]["te_log_ratio_diff"].append(te_log_ratio_diff)
            collectors[name]["ds_score_diff"].append(ds_diff)
            collectors[name]["di_log_ratio"].append(di_lr)
            collectors[name]["ber_diff"].append(ber_diff)

        log(f"📦 Completed fold {fold} in {time.time()-t_fold:.2f}s")

    log("\n🧮 Calculating Wilcoxon p-values across folds …")
    out = {}
    for name, vals in collectors.items():
        t_p = time.time()
        dp_p     = wilcoxon_paired_zero_center(vals["dp_diff"])
        eo_tpr_p = wilcoxon_paired_zero_center(vals["eo_tpr_diff"])
        eo_fpr_p = wilcoxon_paired_zero_center(vals["eo_fpr_diff"])
        eopp_p   = wilcoxon_paired_zero_center(vals["eopp_tpr_diff"])
        fpr_p    = wilcoxon_paired_zero_center(vals["fpr_par_diff"])
        te_p     = wilcoxon_paired_zero_center(vals["te_log_ratio_diff"])
        ds_p     = wilcoxon_paired_zero_center(vals["ds_score_diff"])
        di_p     = wilcoxon_paired_zero_center(vals["di_log_ratio"])
        ber_p    = wilcoxon_paired_zero_center(vals["ber_diff"])
        log(f"   ➡️ {name} p-values computed in {time.time()-t_p:.2f}s")

        out[name] = {
            "Demographic parity": fmt_p(dp_p),
            "Equalised odds": f"TPR={fmt_p(eo_tpr_p)}, FPR={fmt_p(eo_fpr_p)}",
            "Equal opportunity": fmt_p(eopp_p),
            "False positive rate parity": fmt_p(fpr_p),
            "Treatment equality": fmt_p(te_p),
            "Discrimination score": fmt_p(ds_p),
            "Disparate Impact": fmt_p(di_p),
            "Balanced Error Rate": fmt_p(ber_p),
        }
    log("✅ Finished fairness-aware evaluation.")
    return out

def build_table(pvalues_map):
    fairness_rows = [
        "Demographic parity",
        "Equalised odds",
        "Equal opportunity",
        "False positive rate parity",
        "Treatment equality",
        "Discrimination score",
        "Disparate Impact",
        "Balanced Error Rate",
    ]
    model_names = list(pvalues_map.keys())
    data = {m: [pvalues_map[m][row] for row in fairness_rows] for m in model_names}
    return pd.DataFrame(data, index=fairness_rows)

# ===========================
# Run (D7 only)
# ===========================
def main():
    df = load_d7(D7_CFG)
    X, y, g, feats = split_X_y_g(df, D7_CFG["gender_col"], D7_CFG["label_col"])
    log(f"🔹 Features: {len(feats)} | Labels: {np.bincount(y)} | Gender groups: {np.bincount(g)}")

    pvals = evaluate_fairlearn_dp_verbose(X, y, g)
    table = build_table(pvals)

    print("\n📑 Fairness-aware (Demographic Parity) – Wilcoxon p-values table for D7")
    print(table.to_string())
    table.to_csv("D7_fairlearn_DP_pvalues_verbose.csv")
    log("💾 Saved CSV: D7_fairlearn_DP_pvalues_verbose.csv")

if __name__ == "__main__":
    main()

[02:09:04] 📥 Loading D7 …
[02:09:04] ✅ Loaded shape: (70000, 14) (took 0.17s)
[02:09:04] ↪️  Mapped gender [2 1] -> [0 1]
[02:09:04] 🧹 Cleaned shape: (70000, 14)
[02:09:04] 🔹 Features: 12 | Labels: [35021 34979] | Gender groups: [24470 45530]
[02:09:04] 🔹 Starting fairness-aware evaluation (Demographic Parity)
[02:09:04] 
👉 Fold 1/10 — preparing data
[02:09:04]    ✨ Scaled features (took 0.02s)
[02:09:04]    ⏳ Training KNN with fairness constraints …
[02:09:37]    ✅ KNN fitted in 32.59s
[02:09:39]    🔍 KNN predictions in 2.70s
[02:09:42]    🧮 KNN scoring prepared in 2.63s
[02:09:42]    ⏳ Training LR with fairness constraints …
[02:09:56]    ✅ LR fitted in 14.01s
[02:09:56]    🔍 LR predictions in 0.02s
[02:09:56]    🧮 LR scoring prepared in 0.00s
[02:09:56]    ⏳ Training DT with fairness constraints …
[02:10:18]    ✅ DT fitted in 21.68s
[02:10:18]    🔍 DT predictions in 0.01s
[02:10:18]    🧮 DT scoring prepared in 0.00s
[02:10:18]    ⏳ Training RF with fairness constraints …
[02:21:14] 

In [ ]:
# Fairness-Aware ML Evaluation Pipeline (Full Script with p-values)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
from scipy.stats import wasserstein_distance
import warnings

# --- Setup ---
warnings.filterwarnings("ignore")
seed = 42
np.random.seed(seed)

# === Load and clean dataset ===
dataset = pd.read_csv('framingham.csv').dropna()

# --- Simple bias mitigation example (as given): remove 100 male-positive instances ---
male_chd = dataset[(dataset['male'] == 1) & (dataset['TenYearCHD'] == 1)]
if len(male_chd) >= 100:
    dataset = dataset.drop(male_chd.sample(n=100, random_state=seed).index)

# === Prepare data ===
y = dataset['TenYearCHD'].values
X = dataset.drop('TenYearCHD', axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Gender masks
male_mask = dataset['male'] == 1
female_mask = ~male_mask
X_male, X_female = X_scaled[male_mask], X_scaled[female_mask]
y_male, y_female = y[male_mask], y[female_mask]

# === Define models ===
models = {
    'SVM': SVC(probability=True, random_state=seed),
    'LR': LogisticRegression(random_state=seed, max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'RF': RandomForestClassifier(random_state=seed),
    'DT': DecisionTreeClassifier(random_state=seed),
    'ANN': MLPClassifier(random_state=seed, max_iter=500)
}

# === Metric calculation helpers ===
def compute_metrics(y_true, y_pred, y_prob=None):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    if cm.size == 4:
        TN, FP, FN, TP = cm.ravel()
    else:
        TN = FP = FN = TP = 0
    res = {
        'TPR': TP / (TP + FN) if (TP + FN) > 0 else 0.0,
        'TNR': TN / (TN + FP) if (TN + FP) > 0 else 0.0,
        'FPR': FP / (FP + TN) if (FP + TN) > 0 else 0.0,
        'FNR': FN / (TP + FN) if (TP + FN) > 0 else 0.0,
        'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
    }
    if y_prob is not None:
        try:
            res['ROC_AUC'] = roc_auc_score(y_true, y_prob)
        except Exception:
            res['ROC_AUC'] = np.nan
    else:
        res['ROC_AUC'] = np.nan
    return res

# === Group-wise k-fold experiments (original idea, per gender) ===
def run_kfold(X_arr, y_arr, label, n_splits=20, random_state=seed):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(X_arr), start=1):
        X_train, X_test = X_arr[train_idx], X_arr[test_idx]
        y_train, y_test = y_arr[train_idx], y_arr[test_idx]

        fold_result = {'Fold': fold, 'Group': label}
        for name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_prob = None
            if hasattr(model, "predict_proba"):
                y_prob = model.predict_proba(X_test)[:, 1]
            elif hasattr(model, "decision_function"):
                z = model.decision_function(X_test)
                y_prob = 1/(1+np.exp(-z))

            scores = compute_metrics(y_test, y_pred, y_prob=y_prob)
            for k, v in scores.items():
                fold_result[f"{name}_{k}"] = v
        rows.append(fold_result)
    return pd.DataFrame(rows)

print("Running Female Group (20-fold)")
female_df = run_kfold(X_female, y_female, 'Female', n_splits=20, random_state=seed)
print("Running Male Group (20-fold)")
male_df   = run_kfold(X_male, y_male, 'Male', n_splits=20, random_state=seed)
group_kfold_df = pd.concat([female_df, male_df], ignore_index=True)

# === EMD analysis (Table 3-style bias check on labels) ===
def compute_emd_pvalue(df, group_col='male', subgroup_value=0, label_col='TenYearCHD', n_iter=10000, rng=None):
    rng = np.random.default_rng(seed if rng is None else rng)
    all_labels = df[label_col].values
    subgroup = df[df[group_col] == subgroup_value][label_col].values
    observed_emd = wasserstein_distance(subgroup, all_labels)
    boot_emd = []
    for _ in range(n_iter):
        boot_sub = rng.choice(all_labels, size=len(subgroup), replace=False)
        boot_emd.append(wasserstein_distance(boot_sub, all_labels))
    boot_emd = np.array(boot_emd)
    p_val = np.mean(boot_emd >= observed_emd)
    return observed_emd, p_val

female_emd, female_p = compute_emd_pvalue(dataset, subgroup_value=0)
male_emd, male_p     = compute_emd_pvalue(dataset, subgroup_value=1)

emd_summary = pd.DataFrame({
    'Group': ['Female', 'Male'],
    'EMD':   [female_emd, male_emd],
    'p_value': [female_p, male_p],
    'Bias? (p<=0.05)': ['Yes' if female_p <= 0.05 else 'No',
                        'Yes' if male_p <= 0.05 else 'No']
})

print("\nGender Bias EMD Results:")
print(emd_summary)

# === Fairness metrics across gender (train on ALL data; between-group gaps) ===
def safe_rate(numer, denom):
    return numer / denom if denom > 0 else 0.0

def rates(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    if cm.size == 4:
        TN, FP, FN, TP = cm.ravel()
    else:
        TN = FP = FN = TP = 0
    TPR = safe_rate(TP, TP+FN)
    FPR = safe_rate(FP, FP+TN)
    FNR = safe_rate(FN, TP+FN)
    TNR = safe_rate(TN, TN+FP)
    return dict(TP=TP, TN=TN, FP=FP, FN=FN, TPR=TPR, FPR=FPR, FNR=FNR, TNR=TNR)

def fairness_from_groups(y_true, y_pred, y_prob, group_mask):
    """Compute group-wise and disparity metrics (Female=0, Male=1)."""
    g0 = (group_mask == 0)  # Female
    g1 = (group_mask == 1)  # Male

    r0 = rates(y_true[g0], y_pred[g0])
    r1 = rates(y_true[g1], y_pred[g1])

    # Positive prediction rates
    ppr0 = np.mean(y_pred[g0] == 1) if np.any(g0) else 0.0
    ppr1 = np.mean(y_pred[g1] == 1) if np.any(g1) else 0.0

    # Discrimination score (difference in mean positive probability)
    if y_prob is not None:
        dscore = float(np.nan_to_num(np.mean(y_prob[g1]) - np.mean(y_prob[g0])))
    else:
        dscore = float(ppr1 - ppr0)  # fallback

    # Core fairness metrics
    demographic_parity = abs(ppr1 - ppr0)
    disparate_impact   = (ppr0 / ppr1) if ppr1 > 0 else np.nan  # avoid inf -> NaN
    eo_tpr_gap         = abs(r1['TPR'] - r0['TPR'])
    eo_fpr_gap         = abs(r1['FPR'] - r0['FPR'])
    equal_opportunity  = eo_tpr_gap
    fpr_parity         = eo_fpr_gap
    ratio0             = (r0['FN']/r0['FP']) if r0['FP'] > 0 else np.nan
    ratio1             = (r1['FN']/r1['FP']) if r1['FP'] > 0 else np.nan
    treatment_eq       = abs(ratio1 - ratio0) if np.isfinite(ratio0) and np.isfinite(ratio1) else np.nan
    ber0               = 0.5 * (r0['FNR'] + r0['FPR'])
    ber1               = 0.5 * (r1['FNR'] + r1['FPR'])
    ber_mean           = 0.5 * (ber0 + ber1)

    return {
        'Demographic parity (|ΔP(Ŷ=1)|)': demographic_parity,
        'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M))': disparate_impact,
        'Equalised odds |ΔTPR|': eo_tpr_gap,
        'Equalised odds |ΔFPR|': eo_fpr_gap,
        'Equal opportunity |ΔTPR|': equal_opportunity,
        'FPR parity |ΔFPR|': fpr_parity,
        'Treatment equality |Δ(FN/FP)|': treatment_eq,
        'Discrimination score Δ mean ŷ_prob (M−F)': dscore,
        'Balanced Error Rate (mean)': ber_mean,
        'BER_Female': ber0,
        'BER_Male': ber1,
        'PPR_Female': ppr0,
        'PPR_Male': ppr1,
        'TPR_Female': r0['TPR'],
        'TPR_Male': r1['TPR'],
        'FPR_Female': r0['FPR'],
        'FPR_Male': r1['FPR'],
    }

def evaluate_fairness_models(X_all, y_all, group_mask, models_dict, n_splits=5, random_state=seed):
    """Stratified K-fold; compute fairness metrics per fold and average."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []
    for name, clf in models_dict.items():
        fold_metrics = []
        for train_idx, test_idx in skf.split(X_all, y_all):
            Xtr, Xte = X_all[train_idx], X_all[test_idx]
            ytr, yte = y_all[train_idx], y_all[test_idx]
            gte = group_mask[test_idx] if isinstance(group_mask, np.ndarray) else group_mask.iloc[test_idx].values

            clf.fit(Xtr, ytr)
            yhat = clf.predict(Xte)

            prob = None
            if hasattr(clf, "predict_proba"):
                prob = clf.predict_proba(Xte)[:, 1]
            elif hasattr(clf, "decision_function"):
                z = clf.decision_function(Xte)
                prob = 1/(1+np.exp(-z))

            fm = fairness_from_groups(yte, yhat, prob, gte)
            fold_metrics.append(fm)

        avg = pd.DataFrame(fold_metrics).mean(numeric_only=True).to_dict()
        avg['Model'] = name
        rows.append(avg)

    df = pd.DataFrame(rows).set_index('Model').sort_index()
    return df

group_mask_int = dataset['male'].astype(int).values  # 1=Male, 0=Female
fair_df = evaluate_fairness_models(X_scaled, y, group_mask_int, models, n_splits=5, random_state=seed)

# Reorder columns to focus on requested metrics
fairness_summary = fair_df[[
    'Demographic parity (|ΔP(Ŷ=1)|)',
    'Equalised odds |ΔTPR|',
    'Equalised odds |ΔFPR|',
    'Equal opportunity |ΔTPR|',
    'FPR parity |ΔFPR|',
    'Treatment equality |Δ(FN/FP)|',
    'Discrimination score Δ mean ŷ_prob (M−F)',
    'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M))',
    'Balanced Error Rate (mean)',
    'BER_Female', 'BER_Male',
    'PPR_Female', 'PPR_Male',
    'TPR_Female', 'TPR_Male',
    'FPR_Female', 'FPR_Male'
]].copy()

print("\n=== Fairness Summary (avg over 5 folds) ===")
print(fairness_summary.round(3))

# === Permutation tests for p-values on three key metrics ===
def permutation_pvalue(X, y, group_mask, model, metric_key, n_iter=500, random_state=42):
    """
    Compute p-value for a fairness metric via permutation of group labels.
    metric_key is one of:
      'Discrimination score Δ mean ŷ_prob (M−F)',
      'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M))',
      'Balanced Error Rate (mean)'
    """
    rng = np.random.default_rng(random_state)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    # observed
    obs_vals = []
    for tr, te in skf.split(X, y):
        model.fit(X[tr], y[tr])
        yhat = model.predict(X[te])
        prob = None
        if hasattr(model, "predict_proba"):
            prob = model.predict_proba(X[te])[:,1]
        elif hasattr(model, "decision_function"):
            z = model.decision_function(X[te])
            prob = 1/(1+np.exp(-z))
        fm = fairness_from_groups(y[te], yhat, prob, group_mask[te])
        obs_vals.append(fm[metric_key])
    observed = float(np.nanmean(obs_vals))

    # null distribution by shuffling group labels
    null_vals = []
    for _ in range(n_iter):
        shuffled = rng.permutation(group_mask)
        fold_vals = []
        for tr, te in skf.split(X, y):
            model.fit(X[tr], y[tr])
            yhat = model.predict(X[te])
            prob = None
            if hasattr(model, "predict_proba"):
                prob = model.predict_proba(X[te])[:,1]
            elif hasattr(model, "decision_function"):
                z = model.decision_function(X[te])
                prob = 1/(1+np.exp(-z))
            fm = fairness_from_groups(y[te], yhat, prob, shuffled[te])
            fold_vals.append(fm[metric_key])
        null_vals.append(np.nanmean(fold_vals))

    null_vals = np.array(null_vals, dtype=float)

    # For gap metrics (absolute differences), we use a right-tail p-value
    # For DI (a ratio near 1 is "fair"), use two-sided around 1.0
    if metric_key == 'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M))':
        # distance from 1
        obs_d = abs(observed - 1.0)
        null_d = np.abs(null_vals - 1.0)
        p_val = np.mean(null_d >= obs_d)
    else:
        p_val = np.mean(null_vals >= observed)

    return observed, float(p_val)

# Build p-value table for three key metrics
rows = []
metric_keys = [
    'Discrimination score Δ mean ŷ_prob (M−F)',
    'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M))',
    'Balanced Error Rate (mean)'
]

for name, clf in models.items():
    entry = {'Model': name}
    for mk in metric_keys:
        obs, p = permutation_pvalue(X_scaled, y, group_mask_int, clf, mk, n_iter=500, random_state=seed)
        entry[f'{mk}'] = obs
        entry[f'{mk} p-value'] = p
    rows.append(entry)

pval_table = pd.DataFrame(rows).set_index('Model').sort_index()

print("\n=== Key Fairness Metrics with p-values (avg over CV, permutation test) ===")
print(pval_table.round(4))

# === Print compact key metrics to console ===
print("\n=== Key Fairness Metrics (Discrimination, DI, BER) ===")
for model in pval_table.index:
    dscore = pval_table.loc[model, 'Discrimination score Δ mean ŷ_prob (M−F)']
    disp   = pval_table.loc[model, 'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M))']
    ber    = pval_table.loc[model, 'Balanced Error Rate (mean)']
    pdv1   = pval_table.loc[model, 'Discrimination score Δ mean ŷ_prob (M−F) p-value']
    pdv2   = pval_table.loc[model, 'Disparate Impact (P(Ŷ=1|F)/P(Ŷ=1|M)) p-value']
    pdv3   = pval_table.loc[model, 'Balanced Error Rate (mean) p-value']
    print(f"\nModel: {model}")
    print(f"  Discrimination Score : {dscore:.4f}  (p={pdv1:.4f})")
    print(f"  Disparate Impact     : {disp:.4f}  (p={pdv2:.4f})")
    print(f"  Balanced Error Rate  : {ber:.4f}  (p={pdv3:.4f})")

# === Save all outputs into one Excel file ===
out_file = 'results.xlsx'
with pd.ExcelWriter(out_file, mode='w') as xlw:
    group_kfold_df.to_excel(xlw, sheet_name='group_kfold', index=False)
    fairness_summary.round(6).to_excel(xlw, sheet_name='fairness_summary')
    emd_summary.round(6).to_excel(xlw, sheet_name='emd_summary', index=False)
    pval_table.round(6).to_excel(xlw, sheet_name='key_metrics_pvalues')

print(f"\nSaved all results to {out_file} (sheets: group_kfold, fairness_summary, emd_summary, key_metrics_pvalues)")

Running Female Group (20-fold)
Running Male Group (20-fold)

Gender Bias EMD Results:
    Group       EMD  p_value Bias? (p<=0.05)
0  Female  0.005605   0.2631              No
1    Male  0.007490   0.2567              No

=== Fairness Summary (avg over 5 folds) ===
       Demographic parity (|ΔP(Ŷ=1)|)  Equalised odds |ΔTPR|  \
Model                                                          
ANN                             0.022                  0.042   
DT                              0.018                  0.076   
KNN                             0.014                  0.056   
LR                              0.005                  0.037   
RF                              0.009                  0.024   
SVM                             0.004                  0.005   

       Equalised odds |ΔFPR|  Equal opportunity |ΔTPR|  FPR parity |ΔFPR|  \
Model                                                                       
ANN                    0.020                     0.042             